Imports

In [23]:
import os
import pandas as pd
import re
os.getcwd()

'/Users/sergeykorepanov/PycharmProjects/pythonProject'

# Understanding the Dataset

Let's look at the first few rows of the dataset
Even from this small sample we can see a lot of missing data and data that is not relevant.

For example some of the job postings are missing `salary data` which is very important for our project.


In [44]:
df = pd.read_csv('./content/sample_data/us-software-engineer-jobs-zenrows.csv')
df.head()

,title,company,salary,rating,review_count,types,location,relative_time,hires_needed,hires_needed_exact,...,indeed_applyable,ad_id,remote_location,source_id,hiring_event_job,indeed_apply_enabled,job_location_postal,company_overview_link,activity_date,location_extras
0,Android Developer,Shockoe,NaN,0.0,0,Full-time,"Richmond, VA",30+ days ago,NaN,NaN,...,False,NaN,False,11706594,False,False,NaN,NaN,NaN,NaN
1,Web Developer,"Denios, Inc.","$45,000 - $55,000 a year",0.0,0,Full-time,"Louisville, KY",30+ days ago,ONE,1,...,True,370154371.0,False,11468052,False,True,40219.0,NaN,Active 2 days ago,NaN
2,Sr. Android Developer,The Cervantes Group,NaN,4.1,22,Full-time,Remote,6 days ago,TWO_FOUR,2,...,True,NaN,False,9570478,False,True,NaN,/cmp/The-Cervantes-Group,Active 3 days ago,NaN
3,Junior Software Engineer,Medical Knowledge Group,NaN,0.0,0,Full-time,Remote,8 days ago,ONE,1,...,True,368864426.0,False,501562,False,True,NaN,NaN,Active 3 days ago,NaN
4,"Cloud Engineer (Software Engineer Advanced, Ex...",Federal Reserve Bank of New York,NaN,4.1,548,Full-time,"Kansas City, MO",2 days ago,NaN,NaN,...,False,NaN,False,1439,False,False,NaN,/cmp/Federal-Reserve-Bank-of-New-York,NaN,NaN


## Information about data set

From the information we are able to see all type of data in our disposal.
Let list a few of them that are more important to us and describe their meaning



*   **Title** - title of the job position on which company is hiring
*   **Company** - name of the company that is hiring
*   **Salary** - Salary range that the company is offering for the position
*   **Location** - Location where the office of the company is located
*   **Types** - Type of employment: Full-time, Part-time, Contract, Internship, Temporary

Other data points are considered unrelated for our goals. Therefore, let's remove them.



In [45]:
df.info()
columns_to_drop = ['rating', 'review_count', 'relative_time', 'hires_needed', 'hires_needed_exact',
                   'urgently_hiring', 'remote_work_model', 'snippet', 'dradis_job', 'link', 'new_job',
                   'job_link', 'sponsored', 'featured_employer', 'indeed_applyable', 'ad_id', 'remote_location',
                   'source_id', 'hiring_event_job', 'indeed_apply_enabled', 'job_location_postal', 'company_overview_link',
                   'activity_date', 'location_extras']
df_cleaned = df.drop(columns_to_drop, axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58433 entries, 0 to 58432
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  58433 non-null  object 
 1   company                58430 non-null  object 
 2   salary                 18103 non-null  object 
 3   rating                 58433 non-null  float64
 4   review_count           58433 non-null  int64  
 5   types                  42577 non-null  object 
 6   location               58433 non-null  object 
 7   relative_time          58433 non-null  object 
 8   hires_needed           11857 non-null  object 
 9   hires_needed_exact     18439 non-null  object 
 10  urgently_hiring        58433 non-null  bool   
 11  remote_work_model      22804 non-null  object 
 12  snippet                58425 non-null  object 
 13  dradis_job             58433 non-null  bool   
 14  link                   58433 non-null  object 
 15  ne

## Cleaned Data Set
Here is the data remained after cleaning

In [46]:
df_cleaned.head()

,title,company,salary,types,location
0,Android Developer,Shockoe,NaN,Full-time,"Richmond, VA"
1,Web Developer,"Denios, Inc.","$45,000 - $55,000 a year",Full-time,"Louisville, KY"
2,Sr. Android Developer,The Cervantes Group,NaN,Full-time,Remote
3,Junior Software Engineer,Medical Knowledge Group,NaN,Full-time,Remote
4,"Cloud Engineer (Software Engineer Advanced, Ex...",Federal Reserve Bank of New York,NaN,Full-time,"Kansas City, MO"


## Normalising Data
After we cleaned irrelevant data we need to normalise salary ranges. In the code snippet bellow we can see that salary range for a position can be stated as "a year", "a month", "a week", "an hour"

- First of all we will convert salary data to dollars per year.

- And after we will split range data to min_salary and max_salary for the role

### Converting Salary Data
let's look at the unique values in the 'salary' column to understand the data better

In [47]:
# Get unique values after dropping NaNs
unique_salaries = set(df_cleaned['salary'].unique())
unique_salaries_df = pd.DataFrame(unique_salaries)
unique_salaries_df.head(n=100)

,0
0,NaN
1,"$55,000 - $95,000 a year"
2,"$36,000 - $50,000 a year"
3,"$90,000 - $150,000 a year"
4,"$81,585 - $169,166 a year"
...,...
95,"$85,000 - $95,000 a year"
96,"$74,900 - $142,800 a year"
97,"$105,000 - $160,000 a year"
98,"$67,195 - $125,000 a year"


### Converting to Yearly Salary
From previous snippet we see that the salary appears in different formats and rates. 
- a year, a month, a week, an hour
- range and fixed rate

- Let's try to convert all of them to yearly salary

In [48]:
# Cleaning symbols
def clean_text(text: str):
    if pd.isna(text):
        return "None"
    # Split the text by ' - ' to separate the salary range
    cleaned_text = (text
                    .lower()
                    .replace('$', '')
                    .replace(',', '')
                    .replace(' - ', ' '))

    return cleaned_text

df_cleaned['salary_temp'] = df_cleaned['salary'].apply(clean_text)

df_cleaned['salary_temp']

0                      None
1        45000 55000 a year
2                      None
3                      None
4                      None
                ...        
58428                  None
58429                  None
58430                  None
58431                  None
58432                  None
Name: salary_temp, Length: 58433, dtype: object

In [49]:
# Define the assumption for hours worked per year
hours_per_year = 1892  # Statistical data from clockify 

def convert_range_match(match: re.Match[str]) -> object:
    if match.group(3) == 'a year':
        yearly_min = float(match.group(1))
        yearly_max = float(match.group(2))
    if match.group(3) == 'a month':
        yearly_min = float(match.group(1)) * 12
        yearly_max = float(match.group(2)) * 12
    if match.group(3) == 'an hour':
        yearly_min = float(match.group(1)) * hours_per_year
        yearly_max = float(match.group(2)) * hours_per_year        
    return f"{yearly_min:.0f} {yearly_max:.0f}"

def convert_fixed_match(match: re.Match[str]) -> object:
    if match.group(2) == 'a year':
        yearly_min = float(match.group(1))
        yearly_max = yearly_min
    if match.group(2) == 'a month':
        yearly_min = float(match.group(1)) * 12
        yearly_max = yearly_min
    if match.group(2) == 'an hour':
        yearly_min = float(match.group(1)) * hours_per_year
        yearly_max = yearly_min        
    return f"{yearly_min:.0f} {yearly_max:.0f}"

def converter(salary: str):
    if salary == 'None':
        return salary
    
    range_pattern = r'(\d+\.?\d*) (\d+\.?\d*) (a year|a month|an hour)'
    fixed_pattern = r'(\d+\.?\d*) (a year|a month|an hour)'
    
    range_match = re.search(range_pattern, salary)
    fixed_match = re.search(fixed_pattern, salary)
    
    if range_match:
        return convert_range_match(range_match)

    if fixed_match:
        return convert_fixed_match(fixed_match)
# Apply the function to the 'salary' column
df_cleaned['salary_temp'] = df_cleaned['salary_temp'].apply(converter)
df_cleaned['salary_temp']

0               None
1        45000 55000
2               None
3               None
4               None
            ...     
58428           None
58429           None
58430           None
58431           None
58432           None
Name: salary_temp, Length: 58433, dtype: object

In [51]:
# Splitting the 'salary' column on whitespace
salary_split = df_cleaned['salary_temp'].str.split(expand=True)

# Assigning the split parts to 'min_salary' and 'max_salary' columns
df_cleaned['min_salary'] = salary_split[0]
df_cleaned['max_salary'] = salary_split[1]

df_cleaned.drop(columns='salary_temp', inplace=True)
df_cleaned

,title,company,salary,types,location,min_salary,max_salary
0,Android Developer,Shockoe,NaN,Full-time,"Richmond, VA",None,None
1,Web Developer,"Denios, Inc.","$45,000 - $55,000 a year",Full-time,"Louisville, KY",45000,55000
2,Sr. Android Developer,The Cervantes Group,NaN,Full-time,Remote,None,None
3,Junior Software Engineer,Medical Knowledge Group,NaN,Full-time,Remote,None,None
4,"Cloud Engineer (Software Engineer Advanced, Ex...",Federal Reserve Bank of New York,NaN,Full-time,"Kansas City, MO",None,None
...,...,...,...,...,...,...,...
58428,Mobile App Developer,SIDEARM Sports,NaN,NaN,"Syracuse, NY",None,None
58429,Senior Software Engineer (TCAT),Think Surgical,NaN,Full-time,"Fremont, CA",None,None
58430,Software Engineer,"Terma North America, Inc.",NaN,Full-time,"Warner Robins, GA",None,None
58431,Systems Analyst/Programmer,Grant Blackford Mental Health,NaN,Full-time,"Marion, IN",None,None


### Splitting

In [52]:
# df_cleaned.duplicated()
df_cleaned.tail()

,title,company,salary,types,location,min_salary,max_salary
58428,Mobile App Developer,SIDEARM Sports,NaN,NaN,"Syracuse, NY",None,None
58429,Senior Software Engineer (TCAT),Think Surgical,NaN,Full-time,"Fremont, CA",None,None
58430,Software Engineer,"Terma North America, Inc.",NaN,Full-time,"Warner Robins, GA",None,None
58431,Systems Analyst/Programmer,Grant Blackford Mental Health,NaN,Full-time,"Marion, IN",None,None
58432,Senior Engineer II-Software,Microchip Careers,NaN,Full-time,"San Jose, CA",None,None
